In [3]:
# Let's use create a bot with strucutre

In [1]:
# same setup

import os

from dotenv import load_dotenv

load_dotenv()

key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")

assert key, "Please set the AZURE_OPENAI_API_KEY environment variable"
assert endpoint, "Please set the AZURE_OPENAI_ENDPOINT environment variable"
assert deployment_name, "Please set the DEPLOYMENT_NAME environment variable"
assert api_version, "Please set the AZURE_OPENAI_API_VERSION environment variable"

SYS_MSG = "You are a synonym and antonym bot. Given a <word> from the user, you will provide synonyms and antonyms for it."

In [3]:
# LCEL

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    api_version=api_version,  # type: ignore
    azure_deployment=deployment_name,
)

prompt = ChatPromptTemplate.from_messages(
    [("system", SYS_MSG), ("human", "<word>{word}</word>")]
)

chain = prompt | llm

In [4]:
chain.invoke({"word": "excited"})

AIMessage(content='Synonyms for "excited":\n- Thrilled\n- Enthused\n- Eager\n- Animated\n- Elated\n\nAntonyms for "excited":\n- Calm\n- Bored\n- Uninterested\n- Disinterested\n- Indifferent', response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 47, 'total_tokens': 100}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-b912717d-460d-45c4-bff1-2fc5936cc06a-0')

In [5]:
# this is difficult to work with, since it is an AI Message. Let's get to know output parsers that make our life easier.
from langchain_core.output_parsers import StrOutputParser

chain = prompt | llm | StrOutputParser()

chain.invoke({"word": "excited"})

'Synonyms for "excited":\n1. Thrilled\n2. Enthusiastic\n3. Eager\n4. Animated\n5. Ecstatic\n6. Jubilant\n7. Elated\n8. Overjoyed\n9. Delighted\n10. Pumped\n\nAntonyms for "excited":\n1. Calm\n2. Composed\n3. Cool\n4. Collected\n5. Serene\n6. Relaxed\n7. Unexcited\n8. Indifferent\n9. Apathetic\n10. Bored'

In [6]:
# nice! but for building nice apps, we will need some structure

from typing import List

from langchain_core.output_parsers import JsonOutputParser

chain = prompt | llm | JsonOutputParser()

try:
    chain.invoke({"word": "excited"})
except Exception as e:  # noqa: E722 (we know this is bad! :D)
    print("FAILURE! Why?")
    print(e)

FAILURE! Why?
Invalid json output: Synonyms for "excited" include thrilled, ecstatic, elated, enthusiastic, eager, and exhilarated. 

Antonyms for "excited" include calm, composed, indifferent, uninterested, and apathetic.


In [7]:
# our model output is not json formatter. let's adjust the prmopt
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_MSG + "\n. Make sure your output is a valid JSON structure."),
        ("human", "{word}"),
    ]
)

chain = prompt | llm | JsonOutputParser()

chain.invoke({"word": "excited"})

{'word': 'excited',
 'synonyms': ['thrilled', 'enthusiastic', 'eager', 'elated', 'animated'],
 'antonyms': ['calm', 'unenthusiastic', 'indifferent', 'apathetic', 'bored']}

In [11]:
# Nice! But, we can do better. OOP and all that.

from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser


class SynonymAntonym(BaseModel):
    word: str = Field(
        description="The word for which synonyms and antonyms are provided"
    )
    synonyms: List[str] = Field(
        description="List of synonyms for the word, at least one synonym is required"
    )
    antonyms: List[str] = Field(description="List of antonyms for the word, if any")


parser = PydanticOutputParser(pydantic_object=SynonymAntonym)

SYS_MSG = "You are a synonym and antonym bot. Given a <word> from the user, you will provide synonyms and antonyms for it."
# note that we also inject the instructions, so this prompt template is more future proof
prompt = PromptTemplate(
    template=SYS_MSG + "\n{format_instructions}\n{word}\n",
    input_variables=["word"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

In [12]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"word": {"title": "Word", "description": "The word for which synonyms and antonyms are provided", "type": "string"}, "synonyms": {"title": "Synonyms", "description": "List of synonyms for the word, at least one synonym is required", "type": "array", "items": {"type": "string"}}, "antonyms": {"title": "Antonyms", "description": "List of antonyms for the word, if any", "type": "array", "items": {"type": "string"}}}, "required": ["word", "synonyms", "antonyms"]}
```


In [13]:
chain.invoke({"word": "excited"})

SynonymAntonym(word='excited', synonyms=['thrilled', 'enthusiastic', 'eager', 'animated', 'elated'], antonyms=['bored', 'indifferent', 'apathetic', 'unenthusiastic', 'calm'])

In [22]:
from langchain_core.runnables import RunnableLambda
from langchain_core.messages import AIMessage


# now lets force the chain to output something that does not corresponding exactly to the Pydantic model
# this can simulate a model failure to align with the format instructions

prompt = PromptTemplate(
    template=SYS_MSG + "\n{word}\n",
    input_variables=["word"],
)


# adjust the chain so that it prints the output of the llm before the parser
def printer(t: AIMessage):
    print(t.content)
    return t


printer_lambda = RunnableLambda(printer)
chain = prompt | llm | printer_lambda | parser

try:
    chain.invoke({"word": "excited"})
except Exception as e:
    print("fails as expected: ", type(e))

Synonyms for "excited": thrilled, enthusiastic, elated, exhilarated, animated, pumped, ecstatic, eager, roused, stirred.

Antonyms for "excited": calm, composed, indifferent, uninterested, unenthusiastic, bored, disinterested, apathetic, unexcited, subdued.
fails as expected:  <class 'langchain_core.exceptions.OutputParserException'>


In [23]:
from langchain.output_parsers import OutputFixingParser

fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

# remember - the prompt has no format instructions!

chain_w_fix = prompt | llm | fixing_parser

chain_w_fix.invoke({"word": "excited"})

SynonymAntonym(word='excited', synonyms=['thrilled', 'exhilarated', 'enthusiastic', 'elated', 'eager'], antonyms=['calm', 'composed', 'indifferent', 'unenthusiastic', 'bored'])